### This script contains various utility functions that import, clean and transform the data.
Input: Raw data sources
Output: Cleaned data sources ready for furhter analysis

<b>Note:</b> Some column names and specific parts are intentionally excluded for data security reasons.

##### Import Libraries

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

##### Function for data transformation and cleaning

Converting columns to correct data types and renaming columns for easier analysis. Additionally, data is cleaned by dropping inconsistent dates.

In [ ]:
def transform_data(data, emp=False, machine=False, quality=False):
    # Convert all datetime like values to pandas datetime
    date_columns = [###]
    for dt in date_columns:
        if dt in data:
            data[dt] = pd.to_datetime(data[dt])
    if emp:
        data.rename(
            columns={###}, 
                inplace=True)

        data.drop(data.loc[(data.start.dt.year < 2019)].index, inplace=True)
        data.drop(data.loc[(data.end.dt.year < 2019)].index, inplace=True)

    elif machine:
        '''Rename the columns for easier access.'''
        data.rename(
            columns={###},
                inplace=True)

    elif quality:
        data.rename(
            columns={###}, 
                inplace=True)

##### Functions for feature extraction
This function generates new features from date objects.

In [ ]:
def feature_extraction(data, emp=False, quality=False, machine=False):
    if machine:
        data['start_year'] = data['start'].dt.year
        data['end_year'] = data['end'].dt.year
        data['start_month'] = data['start'].dt.month
        data['end_month'] = data['end'].dt.month
        data['start_day'] = data['start'].dt.day_name()
        data['end_day'] = data['end'].dt.day_name()
        data['start_hour'] = data['start'].dt.hour
        data['end_hour'] = data['end'].dt.hour

    elif emp:
        data['start'] = data['start'].dt.round('2H')
        data['end'] = data['end'].dt.round('2H')
        data['start_day'] = data['start'].dt.day_name()
        data['end_day'] = data['end'].dt.day_name()
        data['start_hour'] = data['start'].dt.hour
        data['end_hour'] = data['end'].dt.hour
        data['hours_worked'] = data.end - data.start
        data['hours_worked'] = data['hours_worked'] / np.timedelta64(1, 'h')
        data['hours_worked'] = data['hours_worked'].astype(int)
        data['experience'] = data.end - data.since
        data['experience'] = data['experience'] / np.timedelta64(1, 'Y')
        data['shift'] = data.apply(lambda row: label_shift(row), axis=1)

    elif quality:
        data['start_day'] = data['start'].dt.day_name()
        data['start_hour'] = data['start'].dt.hour

This function extracts a new variable (employee shift) based on the start and end working hours from the shift model (see <b>shift-model.ipynb</b>).

In [ ]:
def label_shift(row):
    if (row['start_day'] == 'Monday') & (row['start_hour'] == 6) & (row['hours_worked'] == 12):
        return 'E12'
    if (row['start_day'] == 'Monday') & (row['start_hour'] == 18) & (row['hours_worked'] == 12):
        return 'N12'
    if (row['start_day'] == 'Tuesday') & (row['start_hour'] == 6) & (row['hours_worked'] == 8):
        return 'E8'
    if (row['start_day'] == 'Tuesday') & (row['start_hour'] == 14) & (row['hours_worked'] == 8):
        return 'L8'
    # other cases are excluded
    return 'No Shift Data'

##### Functions to check for inconsistend dates

In [8]:
def check_date_quality(df):
    cond1 = df.start.dt.year < 2019

    conditions = [cond15, cond13, cond11, cond9, cond7, cond5,
                  cond1, cond3]

    for con in conditions:
        df.drop(df.loc[con].index, inplace=True)

    return df

##### Main functions that trigger the pipeline and produce clean data
Read and clean the employees dataset.

<em> Employee dat

In [9]:
def get_employee_data(clean=True):
    filenames = ['Data_1_RAW.csv', 'Data_2_RAW.csv']
    dataframes = []

    for f in filenames:
        dataframes.append(pd.read_csv(Path().joinpath('data', f), index_col=0))

    if clean:

        for dataframe in dataframes:
            transform_data(dataframe, emp=True)
            feature_extraction(dataframe, emp=True)

        dataframes[0].to_csv(Path().joinpath('data', 'Data_1_CLEAN.csv'))
        dataframes[1].to_csv(Path().joinpath('data', 'Data_2_CLEAN.csv'))

        return dataframes[0], dataframes[1]

    else:
        return dataframes[0], dataframes[1]

Read and clean the industry datasets.

<em>Machine error data:</em> information about machine downtimes, error codes, etc.

<em>Quality control data:</em> information about quality control in the production.

In [10]:
def get_machine_error_data(clean=True):
    dataframe = pd.read_csv(Path().joinpath('data', 'Data_1_RAW.csv'))

    if clean:
        transform_data(dataframe, machine=True)
        feature_extraction(dataframe, machine=True)
        check_date_machine(dataframe)
        dataframe.to_csv(Path().joinpath('data', 'Data_1_CLEAN.csv'))

        return dataframe

    else:
        return dataframe

def get_quality_control_data(clean=True):
    dataframe = pd.read_csv(Path().joinpath('data', 'Data_2_RAW.csv'), index_col=0)

    if clean:
        transform_data(dataframe, quality=True)
        feature_extraction(dataframe, quality=True)
        check_date_quality(dataframe)
        dataframe.to_csv(Path().joinpath('data', 'Data_2_CLEAN.csv'))

        return dataframe

    else:
        return dataframe